## traditional

In [1]:
import numpy as np

def make_cluster(natoms, radius=20, seed=1981):
    np.random.seed(seed)
    arr = np.random.normal(0, radius, size=(natoms,3))-0.5
    return arr

In [2]:
class lj_pure(object):
    
    @classmethod
    def lj(cls, r):
        sr6 = (1./r)**6
        pot = 4.*(sr6*sr6 - sr6)
        return pot


    @classmethod
    def distance(cls, atom1, atom2):
        dx = atom2[0] - atom1[0]
        dy = atom2[1] - atom1[1]
        dz = atom2[2] - atom1[2]

        r = (dx*dx + dy*dy + dz*dz)**0.5
        return r


    @classmethod
    def potential(cls, cluster):
        energy = 0.0
        for i in range(len(cluster)-1):
            for j in range(i+1,len(cluster)):
                r = cls.distance(cluster[i],cluster[j])
                e = cls.lj(r)
                energy += e
        return energy

In [3]:
import numpy as np
class lj_numpy(object):
    
    @classmethod
    def lj(cls, r):
        sr6 = (1./r)**6
        pot = 4.*(sr6*sr6 - sr6)
        return pot
    
    
    @classmethod
    def distances(cls, cluster):
        diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
        mat = np.sqrt((diff*diff).sum(-1))
        return mat

    
    @classmethod
    def potential(cls, cluster):
        d = cls.distances(cluster)
        dtri = np.triu(d)
        energy = cls.lj(dtri[dtri > 1e-4]).sum()
        return energy

In [4]:
cluster = make_cluster(int(2e3), radius=100)

In [5]:
%timeit lj_pure.potential(cluster)

4.44 s ± 64.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit lj_numpy.potential(cluster)

261 ms ± 851 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## numba

In [7]:
import numba

@numba.jit(nopython=True)
def lj(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot


@numba.jit(nopython=True)
def distance(atom1, atom2):
    dx = atom2[0] - atom1[0]
    dy = atom2[1] - atom1[1]
    dz = atom2[2] - atom1[2]

    r = np.sqrt(dx*dx + dy*dy + dz*dz)
    return r


@numba.jit(nopython=True)
def potential(cluster):
    energy = 0.0
    for i in range(len(cluster)-1):
        for j in range(i+1,len(cluster)):
            r = distance(cluster[i],cluster[j])
            e = lj(r)
            energy += e
    return energy

In [8]:
%time potential(np.zeros(shape=(1,3)))

CPU times: user 202 ms, sys: 9.79 ms, total: 211 ms
Wall time: 274 ms


0.0

In [ ]:
cluster = make_cluster(int(2e4), radius=100)
%time lj_numpy.potential(cluster)

In [ ]:
%timeit potential(cluster)

## cuda

In [ ]:
d = lj_numpy.distances(cluster)

In [ ]:
%timeit lj_numpy.distances(cluster)
%timeit lj_numpy.lj(d)

In [ ]:
from numba import cuda
cuda.detect()

In [ ]:
@numba.vectorize(['float64(float64)'], target='cuda')
def cu_lj(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

In [ ]:
%timeit cu_lj(d)

### the easy function first

In [ ]:
def cu_simple(cluster):
    d = lj_numpy.distances(cluster)
    p = cu_lj(d)
    e = np.nansum(p) / 2
    return e

In [ ]:
lj_numpy.potential(cluster)

In [ ]:
%timeit cu_simple(cluster)

In [ ]:
%timeit lj_numpy.potential(cluster)

### try a little harder

In [ ]:
d_device = cuda.to_device(d)

%timeit cu_lj(d_device)

In [ ]:
@cuda.jit('float64(float64)', device=True)
def cu_lj_device(d):
    sr6 = (1./d)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

@cuda.jit
def cu_potential(result, d):
    idx, idy = cuda.grid(2)
    
    if idx < d.shape[0] and idy < d.shape[1]:
        p = cu_lj_device(d[idx, idy])
    
    cuda.atomic.add(result, 0, p)

In [ ]:
import math 

d = lj_numpy.distances(cluster)
d_device = cuda.to_device(d)
result = np.zeros(shape=(1,), dtype=np.float64)

threadsperblock = (8, 8)
blockspergrid_x = math.ceil(d_device.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(d_device.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)

cu_potential[blockspergrid, threadsperblock](result, d_device)


result[0] / 2